In [51]:
from datasets import load_dataset
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

import evaluate
import numpy as np
from transformers import DataCollatorWithPadding
from transformers import DefaultDataCollator

In [52]:
dataset = load_dataset("csv", data_files="AI_Human.csv")
dataset = dataset["train"]

dataset = dataset.shuffle(seed=42).select(range(50000))

split_dataset = dataset.train_test_split(test_size=0.2)
test_valid = split_dataset["test"].train_test_split(test_size=0.5)

final_splits = {
    "train": split_dataset["train"],
    "validation": test_valid["train"],
    "test": test_valid["test"],
}

dataset_dict = final_splits

In [53]:
dataset_dict

{'train': Dataset({
     features: ['text', 'generated'],
     num_rows: 40000
 }),
 'validation': Dataset({
     features: ['text', 'generated'],
     num_rows: 5000
 }),
 'test': Dataset({
     features: ['text', 'generated'],
     num_rows: 5000
 })}

In [54]:
from datasets import DatasetDict
dataset_dict = DatasetDict(dataset_dict)  # Convert it to DatasetDict

In [55]:
dataset_dict = dataset_dict.rename_column("generated", "labels")


In [56]:
dataset_dict['train'].features

{'text': Value(dtype='string', id=None),
 'labels': Value(dtype='float64', id=None)}

In [57]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 40000
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 5000
    })
})

In [58]:
def convert_labels(example):
    example["labels"] = int(example["labels"])
    return example

dataset_dict = dataset_dict.map(convert_labels)


Map: 100%|██████████| 5000/5000 [00:00<00:00, 24739.03 examples/s]


In [59]:
from collections import Counter
print(Counter(dataset_dict["train"]["labels"]))  # Count occurrences of each label

Counter({0.0: 25011, 1.0: 14989})


In [60]:
np.shape(dataset_dict["train"]["labels"])

(40000,)

In [61]:
np.shape(dataset_dict["train"]["text"])

(40000,)

In [62]:
dataset_dict["train"]["text"][0]

"Requiring students a summer project to extend their academic progression is extremely intelligent, however should the summer project be teacher designed or student designed. For me this is simple, letting students design their project would hive them control of their summer the way they desire, could become an amazing social experience, and hive a chance to show their creativity.\n\nTo behind with, the summer is basically one of the few times student he a break from school and no student wants to have to work during it, but allowing them to design their own project hives them the opportunity to work out how and what their going to do and can provide them a much more manageable\xa0schedule. Allowing them to fit in any other activities they wanted to indulged in that summer. Such as, summer camps, road trips, internships, etc. Then both parties are happy and fulfilled with what happened that summer.\n\nSecondly, depending on how the students' ho about the design of their project it coul

In [63]:
model_path = "google-bert/bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_path)

id2label = {0: "Human", 1: "AI"}
label2id = {"Human": 0, "AI": 1}
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2, id2label=id2label, label2id=label2id)
model.config.problem_type = "single_label_classification"

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [64]:
#layers
for name, param in model.named_parameters():
   print(name, param.requires_grad)

bert.embeddings.word_embeddings.weight True
bert.embeddings.position_embeddings.weight True
bert.embeddings.token_type_embeddings.weight True
bert.embeddings.LayerNorm.weight True
bert.embeddings.LayerNorm.bias True
bert.encoder.layer.0.attention.self.query.weight True
bert.encoder.layer.0.attention.self.query.bias True
bert.encoder.layer.0.attention.self.key.weight True
bert.encoder.layer.0.attention.self.key.bias True
bert.encoder.layer.0.attention.self.value.weight True
bert.encoder.layer.0.attention.self.value.bias True
bert.encoder.layer.0.attention.output.dense.weight True
bert.encoder.layer.0.attention.output.dense.bias True
bert.encoder.layer.0.attention.output.LayerNorm.weight True
bert.encoder.layer.0.attention.output.LayerNorm.bias True
bert.encoder.layer.0.intermediate.dense.weight True
bert.encoder.layer.0.intermediate.dense.bias True
bert.encoder.layer.0.output.dense.weight True
bert.encoder.layer.0.output.dense.bias True
bert.encoder.layer.0.output.LayerNorm.weight True


In [65]:
# freeze base model params
for name, param in model.base_model.named_parameters():
    param.requires_grad = False

# unfreeze base model pooling layers
for name, param in model.base_model.named_parameters():
    if "pooler" in name:
        param.requires_grad = True

In [66]:
for name, param in model.named_parameters():
   print(name, param.requires_grad)

bert.embeddings.word_embeddings.weight False
bert.embeddings.position_embeddings.weight False
bert.embeddings.token_type_embeddings.weight False
bert.embeddings.LayerNorm.weight False
bert.embeddings.LayerNorm.bias False
bert.encoder.layer.0.attention.self.query.weight False
bert.encoder.layer.0.attention.self.query.bias False
bert.encoder.layer.0.attention.self.key.weight False
bert.encoder.layer.0.attention.self.key.bias False
bert.encoder.layer.0.attention.self.value.weight False
bert.encoder.layer.0.attention.self.value.bias False
bert.encoder.layer.0.attention.output.dense.weight False
bert.encoder.layer.0.attention.output.dense.bias False
bert.encoder.layer.0.attention.output.LayerNorm.weight False
bert.encoder.layer.0.attention.output.LayerNorm.bias False
bert.encoder.layer.0.intermediate.dense.weight False
bert.encoder.layer.0.intermediate.dense.bias False
bert.encoder.layer.0.output.dense.weight False
bert.encoder.layer.0.output.dense.bias False
bert.encoder.layer.0.output.Lay

In [67]:
import re
import string

def preprocess_and_tokenize(examples):
    preprocessed_texts = []
    for text in examples["text"]:
        text = text.lower()
        text = re.sub(r"\d+", "", text)
        text = text.translate(str.maketrans("", "", string.punctuation))
        text = text.strip()
        preprocessed_texts.append(text)
        
    encoding = tokenizer(preprocessed_texts, padding="max_length", truncation=True)
    encoding["labels"] = [int(label) for label in examples["labels"]]
    return encoding

tokenized_data = dataset_dict.map(preprocess_and_tokenize, batched=True)

Map: 100%|██████████| 5000/5000 [00:02<00:00, 1989.88 examples/s]


In [68]:

accuracy = evaluate.load("accuracy")
auc_score = evaluate.load("roc_auc")

def compute_metrics(eval_pred):
    
    predictions, labels = eval_pred
    
    
    probabilities = np.exp(predictions) / np.exp(predictions).sum(-1, keepdims=True)
    
    positive_class_probs = probabilities[:, 1]
    
    auc = np.round(auc_score.compute(prediction_scores=positive_class_probs, references=labels)['roc_auc'],3)
    
    predicted_classes = np.argmax(predictions, axis=1)
    acc = np.round(accuracy.compute(predictions=predicted_classes, references=labels)['accuracy'],3)
    
    return {"Accuracy": acc, "AUC": auc}

Training Using CUDA

In [69]:
def collate_fn(batch):
        batch = {k: torch.tensor([d[k] for d in batch]) for k in batch[0]}
        batch["labels"] = batch["labels"].long()
        # print(batch)
        return batch
data_collator = DefaultDataCollator()

In [70]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [71]:
lr = 2e-4
batch_size = 50
num_epochs = 5

training_args = TrainingArguments(
    output_dir="bert-ai-classifier_teacher",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)
model.to("cuda")

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [72]:
trainer = Trainer(
    model=model.to("cuda"),
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["validation"],
    tokenizer=tokenizer,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
)


trainer.train()

C:\Users\jai54\AppData\Local\Temp\ipykernel_27148\1966093435.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Auc
1,0.172300,0.127710,0.952000,0.992000
2,0.121900,0.121440,0.952000,0.994000
3,0.115000,0.127628,0.950000,0.995000
4,0.107400,0.106427,0.960000,0.995000
5,0.103800,0.108790,0.959000,0.995000


TrainOutput(global_step=4000, training_loss=0.12408039474487305, metrics={'train_runtime': 7595.1969, 'train_samples_per_second': 26.332, 'train_steps_per_second': 0.527, 'total_flos': 5.2622211072e+16, 'train_loss': 0.12408039474487305, 'epoch': 5.0})

In [73]:
# apply model to validation dataset
predictions = trainer.predict(tokenized_data["train"])

logits = predictions.predictions
labels = predictions.label_ids

metrics = compute_metrics((logits, labels))
print(metrics)

{'Accuracy': np.float64(0.964), 'AUC': np.float64(0.996)}


In [83]:
import torch
from transformers import AutoTokenizer

model.eval()
def aipred(sentence):
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    text = sentence.lower()
    text = re.sub(r"\d+", "", text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    text = text.strip()
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    inputs = {key: val.to("cuda") for key, val in inputs.items()}  # Move tensors to CUDA
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    logits = outputs.logits
    probs = torch.nn.functional.softmax(logits, dim=-1)
    predicted_label = torch.argmax(probs, dim=-1).item()
    print(f"{((probs.data[0][1])*100):.2f}% AI Generated")

In [84]:
aipred("Our country can no longer deliver basic services in times of emergency, as recently shown by the wonderful people of North Carolina — who have been treated so badly — (applause) — and other states who are still suffering from a hurricane that took place many months ago or, more recently, Los Angeles, where we are watching fires still tragically burn from weeks ago without even a token of defense.  They’re raging through the houses and communities, even affecting some of the wealthiest and most powerful individuals in our country — some of whom are sitting here right now.  They don’t have a home any longer.  That’s interesting.  But we can’t let this happen.  Everyone is unable to do anything about it.  That’s going to change")

26.95% AI Generated


In [85]:
aipred(" do not know when it began. This quiet feeling that sits in my chest and refuses to leave. Some days it feels like a shadow, just behind me, always there but never loud. Other days it grows heavy, like I am carrying something invisible that no one else can see.I try to smile. I try to talk. I tell people that I am fine. But deep down, I am just tired. Tired of pretending. Tired of hoping that someone will understand without me having to explain.There is a small part of me that still believes in light, in kindness, in better days. That part keeps me going. That part says, just one more day. Just one more step. And maybe that is enough for now.")

74.59% AI Generated
